# 教學[ckip tagger](https://github.com/ckiplab/ckiptagger/wiki/Chinese-README)

# 1. 資料集給他叫出來

In [23]:
import pandas as pd

df = pd.read_csv("/content/drive/Shareddrives/Media Framing/datasets/0_datasets.csv", encoding = "utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24763 entries, 0 to 24762
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  24763 non-null  object
 1   Link      24763 non-null  object
 2   Date      24763 non-null  object
 3   Source    24763 non-null  object
 4   Category  24763 non-null  object
dtypes: object(5)
memory usage: 967.4+ KB


In [21]:
df.head()

,Headline,Link,Date,Source,Category
0,7成5不滿防疫表現 柯:人在做天在看,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 19:48,華視新聞,politics
1,民調75%網不滿柯防疫表現 柯文哲：人在做天在看,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 17:00,華視新聞,politics
2,航警炫耀「每天打混摸魚」 警署記2大過免職,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 15:25,華視新聞,politics
3,拚紓困4.0三讀 游錫堃:盡最大努力協商,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 14:50,華視新聞,politics
4,澳門停駐台辦事處 陸委會：遺憾澳門單方面做此決定,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 14:20,華視新聞,politics


# 2. 安裝ckiptagger

In [5]:
!pip install -U ckiptagger

In [11]:
# 下載模型 data.zip 到我們當前目錄底下，下方是利用API下載
# from ckiptagger import data_utils, WS
# data_utils.downlaod_data_gdown("./")

In [8]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

In [9]:
import tensorflow
from ckiptagger import WS, POS, NER
# WS = 斷詞, POS = 詞性標註, NER = 實體辨識

In [10]:
# 載入模型
ws = WS("/content/drive/Shareddrives/Fake_news_detector/datasets_fn/ckiptagger_data", disable_cuda=False)
pos = POS("/content/drive/Shareddrives/Fake_news_detector/datasets_fn/ckiptagger_data", disable_cuda=False)
ner = NER("/content/drive/Shareddrives/Fake_news_detector/datasets_fn/ckiptagger_data", disable_cuda=False)

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/usr/local/lib/python3.7/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class

## 開始斷詞

In [14]:
# 將df["Headline"]變成ckip範本一樣的形式:list
# 利用list()就可以把pandas資料轉換型態
sentence_list = list(df["Headline"])
sentence_list

['7成5不滿防疫表現 柯:人在做天在看',
 '民調75%網不滿柯防疫表現 柯文哲：人在做天在看',
 '航警炫耀「每天打混摸魚」 警署記2大過免職',
 '拚紓困4.0三讀 游錫堃:盡最大努力協商',
 '澳門停駐台辦事處 陸委會：遺憾澳門單方面做此決定',
 '殲-16闖我東南空域 專家:威脅花東基地',
 '提案刪藝文紓困經費遭轟！葉毓蘭道歉「宣布撤案」',
 '美國商會肯定政府白皮書議題處理！國發會回應了',
 '苗博雅「全國陪坐牢」挨轟 民眾黨批：對不起選民',
 '朝野要求公開特權名單 蔡壁如也連署',
 '破紀錄！ 共軍出動28架次戰機擾我西南空域',
 '【網路溫度計】一句「失敗人找藉口」狂電蘇貞昌 新一代立法院女戰神是「她」？',
 '124萬劑AZ明開打 雙北預約效率挨轟',
 '未接陳時中電話 黃珊珊:接了有用嗎?',
 '誠懇呼籲85歲長者上網預約打疫苗！柯文哲解釋原因',
 '黃偉哲回總統「粽叛親離」眾歪樓 秒刪改「粽志成城」',
 '紓困4.0預算案 立院明進入實質審查',
 '賈永婕募款捐贈「救命神器」 蔡英文深夜回應了',
 '分析各縣市疫苗施打服務 綠委點名北市「請跟上」',
 'G7公報首提「台海和平重要性」 總統府表達感謝',
 '府：蔡總統未曾說過泰國政府「阻擋」疫苗出口',
 '彭文正指蔡英文已打BNT疫苗 總統府怒駁：不折不扣的謠言',
 '塞爆惹民怨! 交通部:全台14匝道放寬紅燈限制',
 '獲配AZ疫苗少於高雄 雙北市長表不滿',
 '林明溱會勘後摘口罩合照 衛生局開罰',
 '為G7暖身 拜登抵英國開心會強森',
 '7大工業國結盟抗中? 日媒:德義恐被分化',
 '柯稱「唐鳳來不及做好疫苗系統」 唐鳳還原說法打臉',
 '慰留衛生局長 柯文哲：調查有結果、疫情趨緩再處理',
 '很多權貴在好心肝打疫苗？ 柯文哲：掃地歐巴桑更多',
 '國產疫苗還買貴?! 在野轟"不平等合約"',
 '黑箱疫苗爭議 柯文哲遭批甩鍋衛生局',
 '違規接種連環爆 雲林衛生局長道歉',
 '台美貿易官員對話 近期將復談TIFA',
 '好心肝違規 暴露北市疫苗分配沒審查?',
 '疫苗分配風波延燒 柯:數量不夠是事實',
 '稱公會「呱呱叫」 柯文哲：獨厚診所是事實會檢討',
 '驚訝「股長就可決定發疫苗」 柯文哲：以後都公開透明'

In [15]:
word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)

entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [16]:
# 釋放記憶體
del ws
del pos
del ner

In [17]:
# 顯示結果
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return
    
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)

串流輸出內容已截斷至最後 5000 行。
'【賞文迴響】郎英／是願力也是動力'
【(PARENTHESISCATEGORY)　賞文(VA)　迴響(Na)　】(PARENTHESISCATEGORY)　郎英(Nb)　／(FW)　是(SHI)　願力(Na)　也(D)　是(SHI)　動力(Na)　
(6, 8, 'PERSON', '郎英')

'【家人關係】駱展鵬／婆婆是媽媽，兒子是女婿'
【(PARENTHESISCATEGORY)　家人(Na)　關係(Na)　】(PARENTHESISCATEGORY)　駱展鵬(Nb)　／(FW)　婆婆(Na)　是(SHI)　媽媽(Na)　，(COMMACATEGORY)　兒子(Na)　是(SHI)　女婿(Na)　
(6, 9, 'PERSON', '駱展鵬')

'有片！膝蓋卡卡更要動　簡單4招鍛鍊肌力'
有(V_2)　片(Na)　！(EXCLAMATIONCATEGORY)　膝蓋卡卡(Na)　更(Dfa)　要(D)　動(VAC)　　(WHITESPACE)　簡單(VH)　4(Neu)　招(Nf)　鍛鍊(VC)　肌力(Na)　
(5, 7, 'PERSON', '卡卡')
(13, 14, 'CARDINAL', '4')

'【圖輯】圖片故事／綠鬣蜥大暴走 獵蜥達人對戰：利爪刮傷早習以為常'
【(PARENTHESISCATEGORY)　圖輯(Na)　】(PARENTHESISCATEGORY)　圖片(Na)　故事(Na)　／(FW)　綠鬣蜥(Na)　大(VH)　暴走(Na)　 (WHITESPACE)　獵蜥(Na)　達(VJ)　人(Na)　對戰(VA)　：(COLONCATEGORY)　利爪(Na)　刮傷(VC)　早(D)　習以為常(VI)　

'年輕人「不婚不生」沒什麼好驚訝？ 他分析4大關鍵：怎麼敢生'
年輕人(Na)　「(PARENTHESISCATEGORY)　不婚不(D)　生(VC)　」(PARENTHESISCATEGORY)　沒(VJ)　什麼(Nep)　好(Dfa)　驚訝(VK)　？(QUESTIONCATEGORY)　 (WHITESPACE)　他(Nh)　分析(VC)　4(Neu)　大(VH)　關鍵(Na)　：(COLONCATEGORY)　怎麼(D)　敢(VL)　生(VC)　
(20, 21, 'CARDI

# 斷詞結果存到資料集裡

In [35]:
# 計算資料是不是有一樣多
len([word_sentence_list])

1

In [31]:
# 增加ckip_cut欄位
df = pd.DataFrame(data = df, columns=["Headline", "Link", "Date", "Source", "Category"])
df.info()
# df["Link"][2]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24763 entries, 0 to 24762
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  24763 non-null  object
 1   Link      24763 non-null  object
 2   Date      24763 non-null  object
 3   Source    24763 non-null  object
 4   Category  24763 non-null  object
dtypes: object(5)
memory usage: 967.4+ KB


In [ ]:
word_sentence_list

In [ ]:
# 去除標點符號
# import re

# def remove_punctuation(line):
#     rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5]")
#     line = rule.sub('',line)
#     return line
# for li in word_sentence_list:
#     remove_punctuation(li)
#     print(li)

## 如要將原資料集插入新的附欄位名稱的資料，可以使用 **insert()函式**
### 需要留意的是，col的參數只在[0-∞]，因此如果是要抓最後一排的話，可以利用df.shape[1](取得df有幾個欄位)來取得參數

In [38]:
df.insert(5, column="ckip_cut", value=word_sentence_list)
df.head()

,Headline,Link,Date,Source,Category,ckip_cut
0,7成5不滿防疫表現 柯:人在做天在看,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 19:48,華視新聞,politics,"[7成5, 不滿, 防疫, 表現, , 柯, :, 人, 在, 做, 天, 在, 看]"
1,民調75%網不滿柯防疫表現 柯文哲：人在做天在看,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 17:00,華視新聞,politics,"[民調, 75%, 網, 不滿, 柯防疫, 表現, , 柯文哲, ：, 人, 在, 做, ..."
2,航警炫耀「每天打混摸魚」 警署記2大過免職,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 15:25,華視新聞,politics,"[航警, 炫耀, 「, 每, 天, 打混摸魚, 」, , 警署, 記, 2, 大過, 免職]"
3,拚紓困4.0三讀 游錫堃:盡最大努力協商,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 14:50,華視新聞,politics,"[拚, 紓困, 4.0三讀, , 游錫堃, :, 盡, 最, 大, 努力, 協商]"
4,澳門停駐台辦事處 陸委會：遺憾澳門單方面做此決定,https://news.cts.com.tw/cts/politics/202106/20...,2021/06/16 14:20,華視新聞,politics,"[澳門, 停駐, 台, 辦事處, , 陸委會, ：, 遺憾, 澳門, 單方面, 做, 此,..."


In [44]:
df.shape

(24763, 6)

In [47]:
path = "/content/drive/Shareddrives/Media Framing/datasets/"

df.to_csv(path + "1_dataset_ckipcut.csv", encoding = "utf-8", index = False)
